In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

students_math = pd.read_csv("./studentmat.csv")
students_port = pd.read_csv("./studentpor.csv")

In [2]:
students_port.shape

(649, 33)

In [3]:
all_student_rows = [students_math, students_port]

In [4]:
all_students = pd.concat(all_student_rows, ignore_index=True)

In [5]:
all_students.shape

(1044, 33)

In [6]:
all_students
X = all_students[["age", "address", "traveltime", "failures" , "higher",
                "internet", "romantic", "famrel", "freetime", "goout", "absences"]].values

from sklearn.preprocessing import LabelEncoder
discreteCoder_X = LabelEncoder()

In [7]:
X[:,1] = discreteCoder_X.fit_transform(X[:,1])
X[:,4] = discreteCoder_X.fit_transform(X[:,4])
X[:,5] = discreteCoder_X.fit_transform(X[:,5])
X[:,6] = discreteCoder_X.fit_transform(X[:,6])

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

y = all_students[["Walc"]].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=1693)

In [10]:
scale_X = StandardScaler()
X_train = scale_X.fit_transform(X_train)
X_test = scale_X.transform(X_test)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [11]:
from sklearn.svm import SVR

In [12]:
svr_regression = SVR(kernel = "linear", epsilon = 1.0)
svr_regression.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=1.0, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [13]:
#Student A
#Age: 18
#Address: Urban(Label encoded as 1)
#Travel Time: 3 (30 minutes to 1 hour)
#Failures: 3
#Desires for Higher Education: No  (0)
#Internet Access: No (0)
#Romantic Relationship: Yes (1)
#Relationship w/ fam: Ok (2 out of 5)
#Freetime: A lot (5 out of 5)
#Going Out: A bit (2 out of 5)
#Absences: 5

new_studentA = [[18,1,3,3,0,0,1,2,5,2,5]]

In [14]:
new_student_scaledA = scale_X.transform(new_studentA)
studentA_prediction = svr_regression.predict(new_student_scaledA)

In [15]:
studentA_prediction

array([ 2.76152033])

In [16]:
print("First new student (A):" +str(studentA_prediction))

First new student (A):[ 2.76152033]


In [17]:
#Student B
#Age: 18
#Address: Rural(Label encoded as 0)
#Travel Time: 3 (30 minutes to 1 hour)
#Failures: 3
#Desires for Higher Education: No  (0)
#Internet Access: No (0)
#Romantic Relationship: Yes (1)
#Relationship w/ fam: Ok (2 out of 5)
#Freetime: Very little (1 out of 5)
#Going Out: Very little (1 out of 5)
#Absences: 5

new_studentB = [[18,0,3,3,0,0,1,2,1,1,5]]
new_student_scaledB = scale_X.transform(new_studentB)
studentB_prediction = svr_regression.predict(new_student_scaledB)
print("Second new student:"+ str(studentB_prediction))

Second new student:[ 2.31851134]


In [18]:
from sklearn import tree

In [19]:
DT_regression = tree.DecisionTreeRegressor(random_state=1693, max_depth = 3)
DT_regression.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=3, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=1693, splitter='best')

In [20]:
tree.export_graphviz(DT_regression, out_file = "tree.dot", feature_names = ["age", "address",
                                                     "traveltime", "failures" , "higher",
                "internet", "romantic", "famrel", "freetime", "goout", "absences"])

In [21]:
studentA_prediction_RT =DT_regression.predict(new_student_scaledA)
print("Second new student:" + str(studentA_prediction_RT))

Second new student:[ 1.68644068]


In [22]:
studentB_prediction_RT =DT_regression.predict(new_student_scaledB)
print("Second new student:" + str(studentB_prediction_RT))

Second new student:[ 1.68644068]


In [23]:
from sklearn.ensemble import RandomForestRegressor
RF_regression = RandomForestRegressor(n_estimators = 100, random_state=1693)
RF_regression.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=1693, verbose=0, warm_start=False)

In [24]:
studentA_prediction_RF = RF_regression.predict(new_student_scaledA)
print("First new student:" + str(studentA_prediction_RF))

First new student:[ 1.89]


In [25]:
studentB_prediction_RF = RF_regression.predict(new_student_scaledB)
print("First new student:" + str(studentB_prediction_RF))

First new student:[ 1.45]


In [26]:
from sklearn.metrics import mean_absolute_error
rf_MAE = mean_absolute_error(y_test, RF_regression.predict(X_test))

In [27]:
rf_MAE

0.85699797786292031

In [28]:
RT_MAE = mean_absolute_error(y_test, DT_regression.predict(X_test))
SVR_MAE = mean_absolute_error(y_test, svr_regression.predict(X_test))

In [29]:
print("Random Forest MAE: "+ str(rf_MAE))
print("Regression Tree MAE: "+ str(RT_MAE))
print("Support Vector Regression: "+ str(SVR_MAE))

Random Forest MAE: 0.856997977863
Regression Tree MAE: 0.963757325443
Support Vector Regression: 0.961421679225


In [33]:
#Age: 20
#Address: Urban (label encoded as 1)
#Travel Time: 3 (30 minutes to 1 hour)
#Failures: 1
#Desire for Higher Education: No (0)
#Internet Access: Yes (1)
#Romantic Relationship: Yes (1)
#Relationship with Family: Ok (2 out of 5)
#Freetime: A little (3 out of 5)
#Going Out: A bit (2 out of 5)
#Absences: 5

labQue = [[20,1,3,1,0,1,1,2,3,2,5]]
from sklearn.ensemble import RandomForestRegressor
new_student_scaledC = scale_X.transform(labQue)
studentC_prediction = svr_regression.predict(new_student_scaledC)
lab_RF_regression = RandomForestRegressor(n_estimators = 100, random_state=1693)
lab_RF_regression.fit(X_train, y_train)
studentC_prediction_RF = RF_regression.predict(new_student_scaledC)
print("LabQue:" + str(studentC_prediction_RF))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


LabQue:[ 1.77]


In [34]:
#Age: 20
#Address: Urban (label encoded as 1)
#Travel Time: 3 (30 minutes to 1 hour)
#Failures: 1
#Desire for Higher Education: No (0)
#Internet Access: Yes (1)
#Romantic Relationship: Yes (1)
#Relationship with Family: Ok (2 out of 5)
#Freetime: A little (3 out of 5)
#Going Out: A bit (2 out of 5)
#Absences: 5

labQue2 = [[20,1,3,1,0,1,1,2,3,2,5]]
new_student_scaledD = scale_X.transform(labQue2)
studentD_prediction = svr_regression.predict(new_student_scaledD)
print("LabQue2:"+ str(studentD_prediction))


LabQue2:[ 2.50416303]
